# NFL EDA

# import libraries

In [1]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt

# deep learning
import timm
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# loss metrics
from sklearn.metrics import matthews_corrcoef
 
# from torchmetrics.classification import BinaryMatthewsCorrCoef
# matthews_corrcoef = BinaryMatthewsCorrCoef()

# from torchmetrics import MatthewsCorrCoef
# matthews_corrcoef = MatthewsCorrCoef(num_classes=1)#default threshold=0.5

# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")


# Set Configurations

In [2]:
kaggle = False
DEBUG = True
class CFG:
    if kaggle:
        BASE_DIR = "/kaggle/input/nfl-player-contact-detection"
    else:
        BASE_DIR = "/workspace/input"
    TRAIN_HELMET_CSV = os.path.join(BASE_DIR, "train_baseline_helmets.csv")
    TRAIN_TRACKING_CSV = os.path.join(BASE_DIR, "train_player_tracking.csv")
    TRAIN_VIDEO_META_CSV = os.path.join(BASE_DIR, "train_video_metadata.csv")
    TRAIN_LABEL_CSV = os.path.join(BASE_DIR, "train_labels.csv")

    # data config    
    img_size = (224, 224)
    batch_size = 256
    num_workers = 0
    n_fold = 1

    # model config
    model_name = "tf_efficientnet_b0"
    out_features = 1
    inp_channels= 3
    pretrained = True
    
    # learning config
    n_epoch = 20
    lr = 1e-6
    # max_lr = 2e-5
    T_max = 10
    min_lr = 1e-7
    weight_decay = 1e-6
    # opt_wd_non_norm_bias = 0.01
    # opt_wd_norm_bias = 0
    # opt_beta1 = 0.9
    # opt_beta2 = 0.99
    # opt_eps = 1e-5
    
    # validation verbose
    epoch_step_valid = 3
    steps_per_epoch = 50
    
    # etc
    random_seed = 21
    
    if DEBUG:
        n_epoch = 5

# Utils

In [3]:
def seed_everything(seed=CFG.random_seed):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

Using device: cuda


## Dataset Utils

In [4]:
def get_snap_frame(row):
    elaped_time_start2snap = row.snap_time - row.start_time
    elaped_seconds = elaped_time_start2snap.seconds
    snap_frame = elaped_seconds*59.95
    return snap_frame

In [5]:
def set_inimg_window(crop_area, img_size=(720, 1280)):
    left, top, right, bot = crop_area
    # set crop area (area size in img size)
    crop_left = max(0, left)
    crop_top = max(0, top)
    if crop_left != left:   right = right - left
    if crop_top != top:    bot = bot - top
    crop_bot = min(img_size[0], bot)
    crop_right = min(img_size[1], right)
    if crop_bot != bot:    crop_top = crop_top - (bot - img_size[0])
    if crop_right != right:   crop_left = left - (right - img_size[1])
    
    return [crop_left, crop_top, crop_right, crop_bot]

In [6]:
def make_player_mask(img, helmet_pos, img_size=(720, 1280, 3), alpha=0.3):
    crop_size=[-helmet_pos[2], -helmet_pos[3], helmet_pos[2]*3, helmet_pos[3]*6] # helmetの大きさによってplayerの範囲も変更
    base_area = np.array(helmet_pos) + np.array(crop_size) # [left, top, width, height]
    # set players area
    palyer_area = [base_area[0],  base_area[1], base_area[0] + base_area[2], base_area[1] + base_area[3]]
    palyer_area = set_inimg_window(palyer_area)
    mask = np.zeros(img_size, dtype=np.uint8)
    cv2.rectangle(mask, [palyer_area[0], palyer_area[1]], [palyer_area[2], palyer_area[3]], (255, 255, 255), -1)
    mask = np.clip(mask, 0, 1).astype(np.uint8)
    return mask

In [7]:
def get_1player_croparea(helmet_pos, img_size=(720, 1280)):
    crop_size=[-helmet_pos[2]*3, -helmet_pos[3]*4, helmet_pos[2]*6, helmet_pos[3]*6] # helmetの大きさによってplayerの範囲も変更
    players_area = np.array(helmet_pos) + np.array(crop_size) # [left, top, width, height]
    # set players area
    crop_area = [players_area[0],  players_area[1], players_area[0] + players_area[2], players_area[1] + players_area[3]]
    crop_area = set_inimg_window(crop_area)
    return crop_area

In [8]:
def get_2player_croparea(helmet1_pos, helmet2_pos, img_size=(720, 1280)):
    player1_crop_size=[-10, -10, helmet1_pos[2]*4, helmet1_pos[3]*4] # helmetの大きさによってplayerの範囲も変更
    player2_crop_size=[-10, -10, helmet2_pos[2]*4, helmet2_pos[3]*4] # helmetの大きさによってplayerの範囲も変更
    player1_area = np.array(helmet1_pos) + np.array(player1_crop_size) # [left, top, width, height]
    player2_area = np.array(helmet2_pos) + np.array(player2_crop_size) # [left, top, width, height]
    # [left, top, width, height] => [left, top, right, bot]
    player1_crop_area = np.array([player1_area[0], player1_area[1], player1_area[0]+player1_area[2], player1_area[1]+player1_area[3]]) # [left, top, right, bot]
    player2_crop_area = np.array([player2_area[0], player2_area[1], player2_area[0]+player2_area[2], player2_area[1]+player2_area[3]]) # [left, top, right, bot]
    # get min max for set pos in img
    area_min = np.min(np.array([player1_crop_area, player2_crop_area]), axis=0)
    area_max = np.max(np.array([player1_crop_area, player2_crop_area]), axis=0)
    crop_area = [area_min[0], area_min[1], area_max[2], area_max[3]]
    crop_area = set_inimg_window(crop_area)
    return crop_area

In [9]:
def mask_blend(img, mask):
    img = np.clip(img, 0, 255).astype(np.uint8)
    mask = np.clip(mask, 0, 1).astype(np.uint8)
    img = img*mask
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

In [10]:
def search_helmet(helmet_df, player, frame):
    for diff_frame in range(5):
        read_frame = frame + diff_frame
        player_helmet = helmet_df.query('nfl_player_id==@player and frame==@read_frame')
        if len(player_helmet) > 0:
            return player_helmet
        read_frame = frame - diff_frame
        player_helmet = helmet_df.query('nfl_player_id==@player and frame==@read_frame')
        if len(player_helmet) > 0:
            return player_helmet
    return []

# Read Data

In [11]:
helmet_df = pd.read_csv(CFG.TRAIN_HELMET_CSV)
tracking_df = pd.read_csv(CFG.TRAIN_TRACKING_CSV)
videometa_df = pd.read_csv(CFG.TRAIN_VIDEO_META_CSV, parse_dates=["start_time", "end_time", "snap_time"])
target_df = pd.read_csv(CFG.TRAIN_LABEL_CSV, parse_dates=["datetime"])

In [12]:
videometa_df["snap_frame"] = videometa_df.apply(get_snap_frame, axis=1)
helmet_df = helmet_df[['game_play', 'view', 'frame', 'nfl_player_id', 'left', 'width', 'top', 'height']]

In [13]:
print("all game play num = " ,len(target_df["game_play"].unique()))

if not DEBUG:
    train_gameplays = target_df["game_play"].unique()[:50]
    valid_gameplays = target_df["game_play"].unique()[-10:]
else:
    train_gameplays = target_df["game_play"].unique()[:10]
    valid_gameplays = target_df["game_play"].unique()[-3:]

print(len(train_gameplays), len(valid_gameplays))
train_videos = [[gameplay,"Endzone"] for gameplay in train_gameplays]
valid_videos = [[gameplay,"Endzone"] for gameplay in valid_gameplays]

all game play num =  240
10 3


# Dataset

In [14]:
class NFLDataset(Dataset):
    def __init__(self, game_play, view, target_df, helmet_df, meta_df, transform=None):
        self.target_df = target_df
        self.helmet_df = helmet_df
        self.meta_df = meta_df
        self.transform = transform
        self.game_play = game_play
        video_file = game_play + "_" + view + ".mp4"
        self.view = view
        self.video_path = os.path.join(CFG.BASE_DIR, "train", video_file)
        self.cam = cv2.VideoCapture(self.video_path)

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        target = target_info.contact
        game_play1, game_play2, step, player1, player2 = target_info.contact_id.split("_")
        # game_play = game_play1 + "_" + game_play2
        # view = "Endzone"
        meta_info = self.meta_df.query('game_play==@self.game_play and view==@self.view')
        snap_frame = int(meta_info.snap_frame)
        read_frame = snap_frame + int(step) 
        # print(read_frame)
        self.cam.set(cv2.CAP_PROP_POS_FRAMES, int(read_frame))
        ret, img = self.cam.read()
        if ret:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = np.clip(img, 0, 255).astype(np.uint8)
            player1 = int(player1)
            helmet_info = self.helmet_df.query('game_play==@self.game_play and view==@self.view')
            # player1_helmet = helmet_info.query('nfl_player_id==@player1 and frame==@read_frame')
            player1_helmet = search_helmet(helmet_info, player1, read_frame)
            if player2 == "G":
                if len(player1_helmet) > 0:
                    # print("only player1 p2=G")
                    helmet_pos = [player1_helmet.left.values[0], player1_helmet.top.values[0],
                                  player1_helmet.width.values[0], player1_helmet.height.values[0]]
                    crop_area = get_1player_croparea(helmet_pos)
                    mask = make_player_mask(img, helmet_pos)
                    img = mask_blend(img, mask)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]
                else:
                    # print("no player p2=G")
                    pass
            else:
                player2 = int(player2)
                # player2_helmet = helmet_info.query('nfl_player_id==@player2 and frame==@read_frame')
                player2_helmet = search_helmet(helmet_info, player2, read_frame)
                if len(player2_helmet) == 0 and len(player1_helmet)==0:
                    # print("no player len0 p2 not G")
                    pass
                elif len(player1_helmet) > 0 and len(player2_helmet) == 0:
                    # print("only player1 p2notG")
                    helmet_pos = [player1_helmet.left.values[0], player1_helmet.top.values[0],
                                  player1_helmet.width.values[0], player1_helmet.height.values[0]]
                    # img = make_player_mask(img, helmet_pos)
                    mask = make_player_mask(img, helmet_pos)
                    img = mask_blend(img, mask)
                    crop_area = get_1player_croparea(helmet_pos)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]                    
                elif len(player2_helmet) > 0 and len(player1_helmet) == 0:
                    # print("only player2")
                    helmet_pos = [player2_helmet.left.values[0], player2_helmet.top.values[0],
                                  player2_helmet.width.values[0], player2_helmet.height.values[0]]
                    # img = make_player_mask(img, helmet_pos)
                    mask = make_player_mask(img, helmet_pos)
                    img = mask_blend(img, mask)
                    crop_area = get_1player_croparea(helmet_pos)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]
                else:
                    # print("two players")
                    helmet1_pos = [player1_helmet.left.values[0], player1_helmet.top.values[0],
                                  player1_helmet.width.values[0], player1_helmet.height.values[0]]
                    helmet2_pos = [player2_helmet.left.values[0], player2_helmet.top.values[0],
                                  player2_helmet.width.values[0], player2_helmet.height.values[0]]
                    mask1 = make_player_mask(img, helmet1_pos)
                    mask2 = make_player_mask(img, helmet2_pos)
                    mask = mask1 + mask2
                    img = mask_blend(img, mask)
                    crop_area = get_2player_croparea(helmet1_pos, helmet2_pos)
                    img = img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]

            img = cv2.resize(img, dsize=CFG.img_size)
            img = img / 255. # convert to 0-1
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
            target = torch.tensor(target, dtype=torch.float)
            return img, target
        else:
            img = np.zeros(CFG.img_size)
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
            target = torch.tensor(target, dtype=torch.float)
            return img, target

# Model

In [15]:
# without meta
class NFLNet(nn.Module):
    def __init__(
        self,
        model_name = CFG.model_name,
        out_features = CFG.out_features,
        inp_channels= CFG.inp_channels,
        pretrained = CFG.pretrained
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes = out_features)
    
    def forward(self, image):
        output = self.model(image)
        return output

# Loss

In [16]:
# criterion = nn.BCEWithLogitsLoss() #define in train loop

# scheduler

In [17]:
# def divice_norm_bias(model): 
#     norm_bias_params = []
#     non_norm_bias_params = []
#     except_wd_layers = ['norm', '.bias']
#     for n, p in model.model.named_parameters():
#         if any([nd in n for nd in except_wd_layers]):
#             norm_bias_params.append(p)
#         else:
#             non_norm_bias_params.append(p)
#     return norm_bias_params, non_norm_bias_params

In [18]:
# def get_scheduler(optimizer):
#     scheduler = OneCycleLR(
#         optimizer,
#         max_lr=CFG.max_lr,
#         pct_start = 0.25, # same as fastai, defaut 0.3
#         steps_per_epoch=int(((CFG.n_fold - 1) * target_df.shape[0]) / (CFG.n_fold * CFG.batch_size)) + 1,
#         epochs = CFG.n_epoch
#     )
#     return scheduler

# valid fn

In [19]:
# def valid_fn(model, criterion, valid_videos):
#     model.eval()# モデルを検証モードに設定
    
#     valid_dataset = ""
#     for game_play, view in valid_videos:
#         target_game = target_df.query('game_play==@game_play')
#         helmet_game = helmet_df.query('game_play==@game_play and view==@view')
#         videometa_game = videometa_df.query('game_play==@game_play and view==@view')
#         valid_dataset_tmp = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)
#         if len(valid_dataset) == 0:
#             valid_dataset = valid_dataset_tmp
#         else:
#             valid_dataset += valid_dataset_tmp
            
#     valid_loader = DataLoader(
#         valid_dataset,
#         batch_size = CFG.batch_size,
#         shuffle = True,
#         num_workers = CFG.num_workers,
#         pin_memory = True
#     )
    
#     test_targets = []
#     test_preds = []
#     for i, (images, targets) in tqdm(enumerate(valid_loader)):
#         images = images.to(device, non_blocking = True).float()
#         targets = targets.to(device, non_blocking = True).float().view(-1, 1)
#         with torch.no_grad():
#             preds = model(images)
#         loss = criterion(preds, targets)
#         # score = matthews_corrcoef(preds, targets)

#         targets = (targets.detach().cpu().numpy()).ravel().tolist()
#         preds = (torch.sigmoid(preds).detach().cpu().numpy()).ravel().tolist()

#         test_preds.extend(pred)
#         test_targets.extend(targets)
    
#     test_preds = np.array(test_preds)
#     test_targets = np.array(test_targets)
#     del valid_loader, valid_dataset, target, pred
#     gc.collect()
#     torch.cuda.empty_cache()
#     return test_targets, test_preds

In [20]:
def valid_fn(model, criterion, valid_videos):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []
    
    for game_play, view in valid_videos:
        target_game = target_df.query('game_play==@game_play')
        helmet_game = helmet_df.query('game_play==@game_play and view==@view')
        videometa_game = videometa_df.query('game_play==@game_play and view==@view')
        valid_dataset = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)
            
        valid_loader = DataLoader(
            valid_dataset,
            batch_size = CFG.batch_size,
            shuffle = True,
            num_workers = CFG.num_workers,
            pin_memory = True
        )

        for i, (images, targets) in tqdm(enumerate(valid_loader)):
            images = images.to(device, non_blocking = True).float()
            targets = targets.to(device, non_blocking = True).int().view(-1, 1)
            with torch.no_grad():
                preds = model(images)
            loss = criterion(preds, targets)
            # score = matthews_corrcoef(preds, targets)

            targets = targets.detach().cpu().numpy().ravel().tolist()
            preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

            test_preds.extend(pred)
            test_targets.extend(targets)
        
        test_preds = np.array(test_preds)
        test_targets = np.array(test_targets)
        del valid_loader, valid_dataset, target, pred
        gc.collect()
        torch.cuda.empty_cache()
    return test_targets, test_preds

# Train Loop

In [21]:
# def training_loop(train_videos, valid_videos):
#     # oof_df = pd.DataFrame()
#     """ 
#     Set dataset
#     """
#     train_dataset = ""
#     for game_play, view in train_videos:
#         target_game = target_df.query('game_play==@game_play')
#         helmet_game = helmet_df.query('game_play==@game_play and view==@view')
#         videometa_game = videometa_df.query('game_play==@game_play and view==@view')
#         train_dataset_tmp = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)
#         if len(train_dataset) == 0:
#             train_dataset = train_dataset_tmp
#         else:
#             train_dataset += train_dataset_tmp

#     train_loader = DataLoader(
#         train_dataset,
#         batch_size = CFG.batch_size,
#         shuffle = True,
#         num_workers = CFG.num_workers,
#         pin_memory = True
#     )
#     """
#     instantiate model, cost function and optimizer
#     """
#     model = NFLNet()
#     model = model.to(device)
#     criterion = nn.BCEWithLogitsLoss()
#     norm_bias_params, non_norm_bias_params = divice_norm_bias(model)
#     optimizer = torch.optim.AdamW(
#       [
#           {'params': norm_bias_params, 'weight_decay': CFG.opt_wd_norm_bias},
#           {'params': non_norm_bias_params, 'weight_decay': CFG.opt_wd_non_norm_bias},
#       ],
#       betas=(CFG.opt_beta1, CFG.opt_beta2),
#       eps=CFG.opt_eps,
#       lr = CFG.lr,
#       amsgrad = False
#     )
#     scheduler = get_scheduler(optimizer)
#     """
#     train / valid loop
#     """
#     best_rmse = np.inf
#     scaler = GradScaler()
    
#     for epoch in range(1, CFG.n_epoch + 1):
#         print(f'=== epoch: {epoch}: training ===')
        
#         for batch_idx, (images, targets) in enumerate(train_loader):
#             model.train()# modelを学習するモードに設定
#             # 入力データ(image)とラベルデータ(target)をGPUへ
#             images = images.to(device, non_blocking = True).float()
#             targets = targets.to(device, non_blocking = True).float().view(-1, 1)
#             optimizer.zero_grad()
            
#             with autocast(): # mixed precision
#                 preds = model(images)
#                 loss = criterion(preds, targets)
#             score = matthews_corrcoef(preds, targets)
            
#             # 学習率の更新
#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()
#             scheduler.step()
            
#             # 設定したepochでvalidation(検証)データでの予測精度を確認する
#             if ( ( ( batch_idx % CFG.steps_per_epoch == 0) & (epoch >= CFG.epoch_step_valid) ) | ( batch_idx == len(train_loader) ) ):
#                 valid_targets, preds = valid_fn(model, criterion, valid_videos)
#                 valid_score = matthews_corrcoef(preds, targets)
#                 print(f'epoch: {epoch}, batch: {batch_idx}/{len(train_loader)}, valid rmse: {valid_score}')
#                 scheduler.step(valid_score)
                
#                 # validationスコアがbestを更新したらモデルを保存する
#                 if valid_score > best_score:
#                     best_score = valid_score
#                     model_name = CFG.model_path
#                     # torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}_fold{i_fold}.pth')
#                     torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}.pth')
#                     print("saved model.")
#                     # _oof_df = pd.DataFrame(data={'Id': valid_ids, 'pred':preds, 'fold': i_fold, 'Pawpularity':valid_targets}, index=valid_idx)

#     del model, output, train_loader, train_dataset
#     gc.collect()
    
#     torch.cuda.empty_cache()
#     # oof_df = pd.concat([oof_df, _oof_df])
#     # return oof_df.sort_values('Id')

In [22]:
def training_loop(train_videos, valid_videos):
    # oof_df = pd.DataFrame()
    """
    instantiate model, cost function and optimizer
    """
    model = NFLNet()
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    # norm_bias_params, non_norm_bias_params = divice_norm_bias(model)
    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
    
    """ 
    Set dataset
    """
    # train_dataset = ""
    for game_play, view in train_videos:
        print(f"game_play={game_play}, view={view}")
        target_game = target_df.query('game_play==@game_play')
        helmet_game = helmet_df.query('game_play==@game_play and view==@view')
        videometa_game = videometa_df.query('game_play==@game_play and view==@view')
        train_dataset = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)

        train_loader = DataLoader(
            train_dataset,
            batch_size = CFG.batch_size,
            shuffle = True,
            num_workers = CFG.num_workers,
            pin_memory = True
        )

        """
        train / valid loop
        """
        best_score = -np.inf
        scaler = GradScaler()
        
        for epoch in range(1, CFG.n_epoch + 1):
            print(f'=== epoch: {epoch}: training ===')
            
            for batch_idx, (images, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
                model.train()# modelを学習するモードに設定
                # 入力データ(image)とラベルデータ(target)をGPUへ
                images = images.to(device, non_blocking = True).float()
                targets = targets.to(device, non_blocking = True).float().view(-1, 1)
                optimizer.zero_grad()
                
                with autocast(): # mixed precision
                    preds = model(images)
                    loss = criterion(preds, targets)
                    
                targets = targets.detach().cpu().numpy().ravel().tolist()
                preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()
                targets = (np.array(targets) > 0.5).astype(int)
                preds = (np.array(preds) > 0.5).astype(int)
                # score = matthews_corrcoef(torch.sigmoid(preds[:,0]), torch.tensor(targets[:,0], dtype=torch.int))
                score = matthews_corrcoef(targets, preds) # sklearn

                
                # 学習率の更新
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                
                # 設定したepochでvalidation(検証)データでの予測精度を確認する
                if ( ( ( batch_idx % CFG.steps_per_epoch == 0) & (epoch >= CFG.epoch_step_valid) ) | ( batch_idx == len(train_loader) ) ):
                    valid_targets, valid_preds = valid_fn(model, criterion, valid_videos)
                    # valid_score = matthews_corrcoef(valid_preds, valid_targets)
                    valid_score = matthews_corrcoef(valid_targets, valid_preds)
                    print(f'epoch: {epoch}, batch: {batch_idx}/{len(train_loader)}, valid score: {valid_score}')
                    scheduler.step(valid_score)
                    
                    # validationスコアがbestを更新したらモデルを保存する
                    if valid_score > best_score:
                        best_score = valid_score
                        model_name = CFG.model_path
                        # torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}_fold{i_fold}.pth')
                        torch.save(model.state_dict(), f'{CFG.model_dir}/{model_name}.pth')
                        print("saved model.")
                        # _oof_df = pd.DataFrame(data={'Id': valid_ids, 'pred':preds, 'fold': i_fold, 'Pawpularity':valid_targets}, index=valid_idx)

        del model, output, train_loader, train_dataset
        gc.collect()
        
        torch.cuda.empty_cache()
        # oof_df = pd.concat([oof_df, _oof_df])
        # return oof_df.sort_values('Id')

In [23]:
training_loop(train_videos, valid_videos)

game_play=58168_003392, view=Endzone
=== epoch: 1: training ===


  0%|          | 0/70 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# for game_play, view in train_videos:
#         print(f"game_play={game_play}, view={view}")
#         target_game = target_df.query('game_play==@game_play')
#         helmet_game = helmet_df.query('game_play==@game_play and view==@view')
#         videometa_game = videometa_df.query('game_play==@game_play and view==@view')
#         train_dataset = NFLDataset(game_play, view, target_game, helmet_game, videometa_game)

#         train_loader = DataLoader(
#             train_dataset,
#             batch_size = CFG.batch_size,
#             shuffle = True,
#             num_workers = CFG.num_workers,
#             pin_memory = True
#         )        
#         for epoch in range(1, CFG.n_epoch + 1):
#             print(f'=== epoch: {epoch}: training ===')
            
#             for batch_idx, (images, targets) in enumerate(train_loader):
#                 for idx in range(CFG.batch_size):
#                     plt.imshow(images[idx])
#                     plt.title(f"target={targets[idx]}")
#                     plt.show()
#             break